### **Exercise: Sentiment Analysis and Key Insights Extraction from Ford Car Reviews**

### **Problem Statement:**
You have been provided with a dataset containing Ford car reviews. Your task is to use LangChain and the concepts you’ve learned to perform the following tasks:

1. **Sentiment Analysis**: Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and store the result.
2. **Key Insights Extraction**: Extract key pieces of information from each review, such as the pros and cons mentioned, and the specific features the reviewer liked or disliked (e.g., vehicle performance, comfort, price).

You will build a LangChain-based solution that leverages language models to automatically extract this information and provide a structured summary of the reviews. 

---
### **Steps to Solve:**

#### **Step 1: Load the Dataset**
- The dataset file is named `ford_car_reviews.csv` and is sourced from Kaggle: [Edmunds Consumer Car Ratings and Reviews](https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews).
- For this exercise, **limit the data to the first 25 records**. This can be achieved by using `df.head(25)` or `df.iloc[:25]` when loading the data into a DataFrame.

#### **Step 2: Define the Sentiment Analysis Task**
- Use LangChain to create a pipeline to classify the sentiment of each review.
- Define prompts that can guide the model to evaluate the sentiment. For example:
  - "Given the following car review, classify the sentiment as positive, neutral, or negative."

#### **Step 3: Key Insights Extraction**
- Use LangChain to create a pipeline to extract pros, cons, and notable features from each review. Define prompts such as:
  - "What are the pros and cons of the vehicle described in the following review?"
  - "What specific features of the vehicle does the reviewer like or dislike?"

#### **Step 4: Update the DataFrame with New Information**
- Run the pipeline for each review and collect the sentiment and insights.
- Once the analysis and extraction are complete, update the original DataFrame with additional columns to include:
  - Sentiment (positive, neutral, negative)
  - Pros
  - Cons
  - Liked_Features
  - Disliked_Features

---

### **Example Output:**

```json
{
  "Review_Date": "03/07/13",
  "Vehicle_Title": "2006 Ford Mustang Coupe",
  "Review_Text": "With the expected arrival of our 6th child...",
  "Rating": 4.125,
  "Sentiment": "Positive",
  "Pros": "Good driving experience, Large seating capacity, Great options",
  "Cons": "None mentioned",
  "Liked_Features": ["Driving experience", "Seating capacity", "Options available"],
  "Disliked_Features": []
}
```

In [1]:
import pandas as pd
import os, json, re, getpass
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
df=pd.read_csv('ford_car_reviews.csv',engine='python')

In [ ]:
df.drop("Unnamed: 0", axis=1,inplace=True)
df=df.iloc[:25]

In [ ]:
load_dotenv(".env", override=True)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key:")

In [ ]:
model_id = "llama3-8b-8192" #llama3-8b-8192, llama-3.1-8b-instant, llama3-groq-8b-8192-tool-use-preview, llama3-groq-70b-8192-tool-use-preview
llm = ChatGroq(model_name=model_id, temperature=0, )

In [ ]:
combined_system_message_template = """
Given the following car review, perform the following tasks:
1. Classify the sentiment as Positive, Neutral, or Negative.
2. Identify the following details:
   - Pros: List the positive aspects of the vehicle mentioned in the review.
   - Cons: List the negative aspects of the vehicle mentioned in the review.
   - Liked Features: Highlight specific features the reviewer liked (if mentioned).
   - Disliked Features: Highlight specific features the reviewer disliked (if mentioned).
Respond with a JSON object containing these fields: Sentiment, Pros, Cons, Liked_Features, Disliked_Features.
"""
combined_template = ChatPromptTemplate([
    ("system", combined_system_message_template),
    ("human", "{user_input}"),
])

In [ ]:
def analyze_review(review):
    prompt = combined_template.format(user_input=review)
    response = llm.invoke(prompt)
    json_response = JsonOutputParser().invoke(response)
    try:
        result = json_response
    except json.JSONDecodeError:
        result = {
            "Sentiment": "Neutral",  
            "Pros": "Could not extract",
            "Cons": "Could not extract",
            "Liked_Features": [],
            "Disliked_Features": []
        }
    return result



In [ ]:
output_list = []

In [ ]:
for index, row in df.iterrows():
    review_data = {
        "Review_Date": row['Review_Date'],  
        "Vehicle_Title": row['Vehicle_Title'],  
        "Review_Text": row['Review'],  
        "Rating": row['Rating'],  
    }
    analysis = analyze_review(review_data["Review_Text"])
    review_data.update(analysis)
    output_list.append(review_data)

In [ ]:
print(json.dumps(output_list, indent=2))